In [1]:
import pandas as pd

from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn import cross_validation, metrics
from sklearn.metrics import log_loss


import xgboost as xgb
from xgboost import DMatrix

import matplotlib.pyplot as plt
%matplotlib inline

/home/kirill/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
fich = pd.read_csv("fich_mail.csv",",")
res = pd.read_csv("res_mail.csv",",",header=None)
test = pd.read_csv("test_mail.csv",",")

In [3]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(fich,res,
                                                                                     test_size = 0,
                                                                                     random_state = 1234)

In [4]:
dtrain = DMatrix(data=train_data, label=train_labels)

In [5]:
param = {
    'colsample_bytree': 1.0,
    'silent': 1,
    'eval_metric': 'mlogloss',
    'nthread': 6,
    'min_child_weight': 6.0,
    'n_estimators': 285.0,
    'subsample': 0.5, 'eta': 0.001,
    'objective': 'multi:softprob',
    'num_class': 2,
    'max_depth': 3,
    'gamma': 0.9
}
numround = 50000
history = xgb.cv(params=param, dtrain=dtrain, num_boost_round=numround,nfold=5, stratified=True, early_stopping_rounds=200, verbose_eval = 200)

[0]	train-mlogloss:0.692645+2.13542e-06	test-mlogloss:0.692647+6.0531e-06
[200]	train-mlogloss:0.608868+0.000394251	test-mlogloss:0.609198+0.000940304
[400]	train-mlogloss:0.549915+0.000666405	test-mlogloss:0.55053+0.00168737
[600]	train-mlogloss:0.507266+0.000855077	test-mlogloss:0.50813+0.00232583
[800]	train-mlogloss:0.475865+0.00101549	test-mlogloss:0.476969+0.0028343
[1000]	train-mlogloss:0.452451+0.00113743	test-mlogloss:0.453765+0.0032794
[1200]	train-mlogloss:0.434863+0.0012366	test-mlogloss:0.436377+0.00366862
[1400]	train-mlogloss:0.421553+0.0013027	test-mlogloss:0.423255+0.00402375
[1600]	train-mlogloss:0.411464+0.00136595	test-mlogloss:0.413361+0.00431803
[1800]	train-mlogloss:0.403793+0.00141359	test-mlogloss:0.405891+0.00457186
[2000]	train-mlogloss:0.397955+0.00144998	test-mlogloss:0.400257+0.004796
[2200]	train-mlogloss:0.393494+0.00148631	test-mlogloss:0.396003+0.00498394
[2400]	train-mlogloss:0.390078+0.00150898	test-mlogloss:0.392794+0.0051546
[2600]	train-mlogloss:0

In [6]:
history.tail()

,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
11291,0.381141,0.006693,0.367760,0.001759
11292,0.381140,0.006693,0.367759,0.001759
11293,0.381140,0.006694,0.367758,0.001759
11294,0.381140,0.006693,0.367757,0.001759
11295,0.381139,0.006693,0.367756,0.001759


In [7]:
bst = xgb.train(params=param, dtrain=dtrain, num_boost_round=11295)

In [20]:
y_ans = bst.predict(DMatrix(data=train_data.to_sparse()))
train_data = pd.concat([train_data , pd.DataFrame(y_ans)[1] , train_labels] , axis = 1)
train_data = train_data.rename_axis({0: "res"}, axis="columns")
train_data = train_data.rename_axis({1: "preds"}, axis="columns")

In [21]:
train_data["error"] = abs(train_data.res - train_data.preds)
train_data = train_data[train_data.error < 0.97]
train_labels = train_data.res
train_data = train_data.drop("res" , axis = 1)
train_data = train_data.drop("preds" , axis = 1)
train_data = train_data.drop("error" , axis = 1)

### Здесь надо занаво обучить алгоритм 

In [13]:
test

,maxPlayerLevel,numberOfAttemptedLevels,attemptsOnTheHighestLevel,totalNumOfAttempts,averageNumOfTurnsPerCompletedLevel,doReturnOnLowerLevels,numberOfBoostersUsed,fractionOfUsefullBoosters,totalScore,totalBonusScore,...,maxPlayerLevel_cluster,numberOfAttemptedLevels_cluster,attemptsOnTheHighestLevel_cluster,totalNumOfAttempts_cluster,averageNumOfTurnsPerCompletedLevel_cluster,numberOfBoostersUsed_cluster,totalBonusScore_cluster,totalStarsCount_cluster,numberOfDaysActuallyPlayed_cluster,totalScore_cluster
0,0.546258,0.253664,0.002747,0.189107,0.782542,0.0,0.390511,1.000000,0.460614,0.455537,...,0,0,0,0,1,0,0,0,0,1
1,0.378610,0.167691,0.000000,0.107001,0.638943,0.0,0.000000,0.000000,0.362750,0.430821,...,0,0,0,0,0,0,0,0,0,0
2,0.732359,0.276018,0.005495,0.313442,0.847084,0.0,0.491041,0.400000,0.501321,0.478974,...,0,0,0,0,1,0,0,0,1,1
3,0.718916,0.200538,0.008242,0.232190,0.859948,0.0,0.000000,0.000000,0.444153,0.397159,...,0,0,1,0,1,0,0,0,1,0
4,0.450246,0.253664,0.000000,0.161860,0.696341,0.0,0.390511,1.000000,0.444071,0.478974,...,0,0,0,0,0,0,0,0,0,0
5,0.643275,0.253664,0.002747,0.232190,0.789690,0.0,0.552265,0.500000,0.462950,0.450389,...,0,0,0,0,1,0,0,0,0,1
6,0.546258,0.378259,0.002747,0.319307,0.748631,1.0,0.552265,0.625000,0.520101,0.546187,...,0,0,0,0,0,0,0,0,1,1
7,0.779848,0.697652,0.002747,0.541717,0.800306,0.0,0.666788,0.764706,0.796913,0.781103,...,0,0,0,0,1,0,1,0,1,1
8,0.523914,0.348654,0.000000,0.222472,0.737263,0.0,0.464398,1.000000,0.491203,0.521007,...,0,0,0,0,0,0,0,0,0,1
9,0.287681,0.076599,0.000000,0.145975,0.500000,1.0,0.000000,0.000000,0.350715,0.312767,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_ans = pd.DataFrame(bst.predict(DMatrix(data=test.to_sparse())))[1]
y_ans.head()

0    0.117315
1    0.028941
2    0.612046
3    0.620943
4    0.055980
Name: 1, dtype: float32

In [15]:
df_to_save = pd.DataFrame(y_ans)

In [17]:
df_to_save.index = test.index

In [18]:
for ind in test_labels[test_data.maxPlayerLevel == 0].index:
    df_to_save[df_to_save.index == ind] = 0.0326

In [19]:
df_to_save.to_csv('preds_xgb_001.csv', sep = ',', index=False,header=None)